In [15]:
# imports
from astropy.io import fits
# import matplotlib.pyplot as plt
import numpy as np
import csv 
import match

In [2]:
# path variables 
path_cat = 'D:\IfA REU\Catalogs'
path_fig = 'D:\IfA REU\IfA-REU-Project\Figures'
path_csv = 'D:\IfA REU\IfA-REU-Project\Lists'

# Get Catalog Data 

In [3]:
# open COSMOS 2016 catalog file and get data 
inf16 = fits.open('D:\IfA REU\Catalogs\chandra_COSMOS_legacy_opt_NIR_counterparts_20160113_4d.fits')
data16 = inf16[1].data
cols16 = inf16[1].columns
inf16.close()

In [4]:
# print(cols16)

In [5]:
# open COSMOS 2020 catalog file and get data 
inf20 = fits.open(path_cat+'\COSMOS2020_CLASSIC_R1_v2.0_master.fits')
data20 = inf20[1].data
cols20 = inf20[1].columns
inf20.close()

In [6]:
# print(cols20)

In [7]:
# get id columns
id16_data = np.array(data16['id_k_uv '])
id20_data = np.array(data20['ID_COSMOS2015'])

In [8]:
##### get the ID lists from the Donley 2012 selected AGN csv file

# initialize list
id_irAGN_csv = []

# open csv file
with open(path_csv+'\COSMOS2020_DONLEY2012_AGN.csv', newline='') as csv_file : 
    # read file 
    reader = csv.reader(csv_file, delimiter=',', quotechar='"')
    # output file into list for easy access
    for row in reader :
        id_irAGN_csv.append(row)
    #print description
    print(id_irAGN_csv[0])

# save important IDs 
AGNid_2020 = np.asarray(id_irAGN_csv[2],dtype=int)
print('First ID: ', AGNid_2020[0])
print(type(AGNid_2020[0]))

['This file contains the IDs of Donley et al 2012 selected AGN from the COSMOS 2020 catalog. These lists were generated in COSMOS20_AGN_selection.ipynb. Line 1 is the ID, Line 2 is the ID_COSMOS2015.']
First ID:  44036
<class 'numpy.int32'>


In [29]:
# print out info
print('Col \t\t Min \t\t Max \t\t Count')
print('-------------------------------------------------------------')
print('ID 2016 \t',         min(id16_data), '\t\t',   max(id16_data), '\t',    len(id16_data))
print('ID 2020 \t',         min(id20_data), '\t\t',   max(id20_data), '\t',    len(id20_data))
print('DONLEY 2020 ID\t',   min(AGNid_2020),'\t\t',   max(AGNid_2020),'\t',    len(AGNid_2020))

Col 		 Min 		 Max 		 Count
-------------------------------------------------------------
ID 2016 	 -99 		 1146373 	 4016
ID 2020 	 -99 		 1182096 	 1720700
DONLEY 2020 ID	 7793 		 1162566 	 2287


# Match Catalogs

In [10]:
# returns the index of the match
def FindMatch(search, dataList) :
    # search list 
    for i, value in enumerate(dataList) :
        # if the two ids match,
        if(search == value) : 
            # return current index
            return(i)      
    # returns -99 if there is no match.
    return(-99)

In [11]:
# ##### match ID test

# intitialize lists 
id_matchKey = []
test = [1,2,3]
t1 = [0,2,0]
t2 = [1,0,3]

# for each Donley agn
for id in test : 

    # find the index
    i16 = FindMatch(id, t1)
    i20 = FindMatch(id, t2)

    # ad id and location index to the key list 
    id_matchKey.append([id, i16, i20])

    # progress
    print([id, i16, i20])


[1, -99, 0]
[2, 1, -99]
[3, -99, 2]


In [74]:
# intitialize lists 
test = np.array([55,22,33,44,11])
t1 = np.array([55,11,22,99,99])

res1, res2 = match.match(t1, test)

t1_match = t1[res1]
test_match = test[res2]


# print('ID: ', t1_match)
# print('i:  ', res1)
# print("====")
# print('ID: ', test_match)
# print('i:  ', res2)

print('ID', 'i')
print(t1_match[0], res1[0] )
print(t1_match[1], res1[1] )
print(t1_match[2], res1[2] )



ID i
11 1
22 2
55 0


In [61]:
# ###### match the given ID to the ID_COSMOS2015 in the 2020 catalog and the id_k_uv in the 2016 catalog
# # This block takes 51min to run

# # open csv file to write to 
# with open(path_csv+'\COSMOS_match_DONLEY2012_AGN.csv', mode='w', newline='') as csv_file:
#     writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)

#     # write column names 
#     writer.writerow(['ID','2016','2020'])
    
#     # for each Donley agn ID, search for the index of the matching ID
#     for id in AGNid_2020: 
#         # find the index
#         i16 = FindMatch(id, id16_data)
#         i20 = FindMatch(id, id20_data)
#         # write id and location index 
#         writer.writerow([id, i16, i20])

In [77]:
###### Match AGN ID (from COSMOS2020 id='ID_CHANDRA2015) to the COSMOS 2020 catalog (id='ID_CHANDRA2015) and get the index of matches

# match 
agn20_a, agn20_b = match.match(id20_data, AGNid_2020)

# get i
id20_match20 = id20_data[agn20_a]

with open(path_csv+'\Match_AGNID_COSMOS2020.csv', mode='w', newline='') as csv_file:
    writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)

    # write column names 
    writer.writerow(['ID','2020'])

    # fill table 
    for i, v in enumerate(id20_match20) :
        writer.writerow([v, agn20_a[i]])

In [78]:
###### Match AGN ID (from COSMOS2020 id='ID_CHANDRA2015) to the Chandra 2016 catalog (id='id_k_uv) and get the index of matches

# match 
agn16_a, agn16_b = match.match(id16_data, AGNid_2020)

# get i 
id16_match16 = id16_data[agn16_a]

with open(path_csv+'\Match_AGNID_Chandra2016.csv', mode='w', newline='') as csv_file:
    writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)

    # write column names 
    writer.writerow(['ID','2016'])

    # fill table 
    for i, v in enumerate(id16_match16) :
        writer.writerow([v, agn16_a[i]])

In [80]:
print('AGN matches in COSMOS 2020:\t',  len(id20_match20))
print('AGN matches in Chandra 2016:\t', len(id16_match16))

AGN matches in COSMOS 2020:	 2287
AGN matches in Chandra 2016:	 746


# TO DO

1. get "ID_CHANDRA=" AGN IR list from COSMOS 2020 
2. match 2020 to 2016 catalog (get IRAC data)   <--- HERE right now
3. select AGN using XRAYS
4. get list of ids
5. match IR and XRAY IDs
6. Plot IR and XRAY selected AGN on same plot using IRAC filters (i.e. IR blue dot, XRAY red x)

